In [1]:
import datetime
import time
from ib_insync import *
import pandas_ta as ta
import pandas as pd
util.startLoop()
ib = IB()

ib.connect('127.0.0.1', 7497, clientId=207)

tickers=['SBIN','KOTAKBANK']
contract_objects={}
for tick in tickers:
    print(Stock(tick,'NSE','INR'))
    contract_objects[tick]=ib.qualifyContracts(Stock(tick,'NSE','INR'))[0]
print(contract_objects)

Stock(symbol='SBIN', exchange='NSE', currency='INR')
Stock(symbol='KOTAKBANK', exchange='NSE', currency='INR')
{'SBIN': Stock(conId=173418084, symbol='SBIN', exchange='NSE', primaryExchange='NSE', currency='INR', localSymbol='SBIN', tradingClass='SBIN'), 'KOTAKBANK': Stock(conId=78743661, symbol='KOTAKBANK', exchange='NSE', primaryExchange='NSE', currency='INR', localSymbol='KOTAKBANK', tradingClass='KOTAKBANK')}


In [6]:
def stockprice(temp):
    a=ib.reqMktData(temp)
    util.sleep(1)
    price=a.last
    ib.cancelMktData(temp)
    return price

In [9]:
stockprice(Crypto('BTC', 'PAXOS', 'USD'))

42353.5

# Get historical data for ticker along with SMA

In [11]:
def get_historical_data(ticker):
    ticker_contract = contract_objects[ticker]
    bars = ib.reqHistoricalData(
    ticker_contract, endDateTime='', durationStr='10 D',
    barSizeSetting='1 min', whatToShow='TRADES', useRTH=True)
    # convert to pandas dataframe:
    df = util.df(bars)
    print(df)
    df['sma1']=ta.sma(df.close,20)
    df['sma2']=ta.sma(df.close,50)
    return df

In [12]:
get_historical_data('SBIN')

                          date    open    high     low   close  volume  \
0    2024-01-19 09:15:00+05:30  632.85  633.25  632.55  632.55  1097.0   
1    2024-01-19 09:16:00+05:30  633.00  633.00  631.95  632.35  3875.0   
2    2024-01-19 09:17:00+05:30  632.40  632.95  632.15  632.90  3013.0   
3    2024-01-19 09:18:00+05:30  633.15  633.20  631.35  631.85  2849.0   
4    2024-01-19 09:19:00+05:30  631.85  632.30  631.55  631.80  2856.0   
...                        ...     ...     ...     ...     ...     ...   
3745 2024-02-05 15:25:00+05:30  642.70  642.70  642.25  642.40  9405.0   
3746 2024-02-05 15:26:00+05:30  642.30  642.50  642.20  642.30  5391.0   
3747 2024-02-05 15:27:00+05:30  642.25  642.65  642.25  642.40  2618.0   
3748 2024-02-05 15:28:00+05:30  642.45  642.80  642.35  642.55  2327.0   
3749 2024-02-05 15:29:00+05:30  642.55  643.45  642.40  642.50  2446.0   

      average  barCount  
0     632.635         9  
1     632.230        56  
2     632.435        53  
3     6

,date,open,high,low,close,volume,average,barCount,sma1,sma2
0,2024-01-19 09:15:00+05:30,632.85,633.25,632.55,632.55,1097.0,632.635,9,NaN,NaN
1,2024-01-19 09:16:00+05:30,633.00,633.00,631.95,632.35,3875.0,632.230,56,NaN,NaN
2,2024-01-19 09:17:00+05:30,632.40,632.95,632.15,632.90,3013.0,632.435,53,NaN,NaN
3,2024-01-19 09:18:00+05:30,633.15,633.20,631.35,631.85,2849.0,632.680,60,NaN,NaN
4,2024-01-19 09:19:00+05:30,631.85,632.30,631.55,631.80,2856.0,631.885,58,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
3745,2024-02-05 15:25:00+05:30,642.70,642.70,642.25,642.40,9405.0,642.365,59,643.1400,642.978
3746,2024-02-05 15:26:00+05:30,642.30,642.50,642.20,642.30,5391.0,642.355,55,643.0775,642.938
3747,2024-02-05 15:27:00+05:30,642.25,642.65,642.25,642.40,2618.0,642.425,57,643.0000,642.899
3748,2024-02-05 15:28:00+05:30,642.45,642.80,642.35,642.55,2327.0,642.445,55,642.9175,642.880


# PLACE MARKET BUY AND STOP(SELL) ORDER FOR TICKER

In [54]:
def trade_buy_stocks(ticker,closing_price=0,quantitys=1):
    #market order
    contract = contract_objects[ticker]
    ord=MarketOrder(action='BUY',totalQuantity=quantitys)
    trade=ib.placeOrder(contract,ord)
    ib.sleep(1)
    print(trade)

    #stop loss order
    # limit stop
    ord=StopOrder(action='SELL',totalQuantity=quantitys,stopPrice=int(0.98*closing_price))
    trade=ib.placeOrder(contract,ord)
    ib.sleep(1)
    print(trade)

    # trailing stop
    # order = Order()
    # order.orderId = ib.client.getReqId()
    # order.action = 'SELL'
    # order.orderType = "TRAIL"
    # order.totalQuantity = 1
    # order.trailingPercent = 2
    # order.trailStopPrice = int(stock_price*0.98)
    # trade=ib.placeOrder(contract, order)
    # ib.sleep(1)

In [58]:
trade_buy_stocks('SBIN',642)

Trade(contract=Stock(conId=173418084, symbol='SBIN', exchange='NSE', primaryExchange='NSE', currency='INR', localSymbol='SBIN', tradingClass='SBIN'), order=MarketOrder(orderId=36, clientId=207, permId=90090868, action='BUY', totalQuantity=1.0, lmtPrice=0.0, auxPrice=0.0), orderStatus=OrderStatus(orderId=36, status='PreSubmitted', filled=0.0, remaining=1.0, avgFillPrice=0.0, permId=90090868, parentId=0, lastFillPrice=0.0, clientId=207, whyHeld='', mktCapPrice=0.0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2024, 2, 5, 21, 41, 45, 679550, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='', errorCode=0), TradeLogEntry(time=datetime.datetime(2024, 2, 5, 21, 41, 46, 37277, tzinfo=datetime.timezone.utc), status='PreSubmitted', message='', errorCode=0)], advancedError='')
Trade(contract=Stock(conId=173418084, symbol='SBIN', exchange='NSE', primaryExchange='NSE', currency='INR', localSymbol='SBIN', tradingClass='SBIN'), order=StopOrder(orderId=37, clientId=207, permId=

# Place sell order and stop order

In [51]:
def trade_sell_stocks(ticker,stock_price,quantities=1):
    #market order
    contract = contract_objects[ticker]
    ord=MarketOrder(action='SELL',totalQuantity=quantities)
    trade=ib.placeOrder(contract,ord)
    ib.sleep(1)
    print(trade)

    # trailing stop loss order

    ord=StopOrder(action='BUY',totalQuantity=quantities,stopPrice=int(1.02*stock_price))
    trade=ib.placeOrder(contract,ord)
    ib.sleep(1)
    print(trade)

    # order = Order()
    # order.orderId = ib.client.getReqId()
    # order.action = 'BUY'
    # order.orderType = "TRAIL"
    # order.totalQuantity = 1
    # order.trailingPercent = 2
    # order.trailStopPrice = int(stock_price*1.02)
    # trade=ib.placeOrder(contract, order)
    # ib.sleep(1)


In [52]:
trade_sell_stocks('KOTAKBANK',1800)

Trade(contract=Stock(conId=78743661, symbol='KOTAKBANK', exchange='NSE', primaryExchange='NSE', currency='INR', localSymbol='KOTAKBANK', tradingClass='KOTAKBANK'), order=MarketOrder(orderId=32, clientId=207, permId=90090864, action='SELL', totalQuantity=1.0, lmtPrice=0.0, auxPrice=0.0), orderStatus=OrderStatus(orderId=32, status='PreSubmitted', filled=0.0, remaining=1.0, avgFillPrice=0.0, permId=90090864, parentId=0, lastFillPrice=0.0, clientId=207, whyHeld='', mktCapPrice=0.0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2024, 2, 5, 21, 40, 3, 811195, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='', errorCode=0), TradeLogEntry(time=datetime.datetime(2024, 2, 5, 21, 40, 4, 171841, tzinfo=datetime.timezone.utc), status='PreSubmitted', message='', errorCode=0)], advancedError='')
Trade(contract=Stock(conId=78743661, symbol='KOTAKBANK', exchange='NSE', primaryExchange='NSE', currency='INR', localSymbol='KOTAKBANK', tradingClass='KOTAKBANK'), order=StopOrder(order

In [75]:
contract =ib.qualifyContracts(Stock('SPY','SMART','USD'))[0]
print(contract)
ord=MarketOrder(action='BUY',totalQuantity=1)
trade=ib.placeOrder(contract,ord)
ib.sleep(1)
print(trade)

Stock(conId=756733, symbol='SPY', exchange='SMART', primaryExchange='ARCA', currency='USD', localSymbol='SPY', tradingClass='SPY')
Trade(contract=Stock(conId=756733, symbol='SPY', exchange='SMART', primaryExchange='ARCA', currency='USD', localSymbol='SPY', tradingClass='SPY'), order=MarketOrder(orderId=54, clientId=207, permId=90090885, action='BUY', totalQuantity=1.0, lmtPrice=0.0, auxPrice=0.0), orderStatus=OrderStatus(orderId=54, status='PreSubmitted', filled=0.0, remaining=1.0, avgFillPrice=0.0, permId=90090885, parentId=0, lastFillPrice=0.0, clientId=207, whyHeld='', mktCapPrice=0.0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2024, 2, 5, 21, 53, 18, 479140, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='', errorCode=0), TradeLogEntry(time=datetime.datetime(2024, 2, 5, 21, 53, 19, 159315, tzinfo=datetime.timezone.utc), status='PreSubmitted', message='', errorCode=0)], advancedError='')


# Close all open orders for a ticker

In [34]:
def close_ticker_open_orders(ticker):
    ord=ib.reqAllOpenOrders()
    df1=util.df(ord)
    df1['ticker_name']=[cont.symbol for cont in df1['contract']]
    for i in range(len(df1)):
        order_object=df1[df1['ticker_name']==ticker].order.iloc[i]
        print(order_object)
        ib.cancelOrder(order_object)

In [59]:
close_ticker_open_orders('SBIN')

MarketOrder(orderId=36, clientId=207, permId=90090868, action='BUY', totalQuantity=1.0, lmtPrice=0.0, auxPrice=0.0)
StopOrder(orderId=37, clientId=207, permId=90090869, action='SELL', totalQuantity=1.0, lmtPrice=0.0, auxPrice=629.0)


In [53]:
close_ticker_open_orders('KOTAKBANK')

MarketOrder(orderId=32, clientId=207, permId=90090864, action='SELL', totalQuantity=1.0, lmtPrice=0.0, auxPrice=0.0)
StopOrder(orderId=33, clientId=207, permId=90090865, action='BUY', totalQuantity=1.0, lmtPrice=0.0, auxPrice=1836.0)


# Close all open positions for a ticker

In [3]:
df2=util.df(ib.reqPositions())
df2

,account,contract,position,avgCost
0,DU8498783,"Crypto(conId=479624278, symbol='BTC', currency...",0.023613,42426.278787


In [4]:
def close_ticker_position(name):

    df2=util.df(ib.reqPositions())
    df2['ticker_name']=[cont.symbol for cont in df2['contract']]
    cont=df2[df2['ticker_name']==name].contract.iloc[0]
    quant=df2[df2['ticker_name']==name].position.iloc[0]
    quant
    if quant>0:
        #sell
        ord=MarketOrder(action='SELL',totalQuantity=quant)
        ib.placeOrder(cont,ord)
    elif quant<0:
        ord=MarketOrder(action='BUY',totalQuantity=abs(quant))
        ib.placeOrder(cont,ord)


In [ ]:
close_ticker_position('BTC')

Error 321, reqId 60: Error validating request.-'bN' : cause - Missing order exchange.
Canceled order: Trade(contract=Crypto(conId=479624278, symbol='BTC', currency='USD', localSymbol='BTC.USD', tradingClass='BTC'), order=MarketOrder(orderId=60, clientId=207, action='SELL', totalQuantity=0.02361261), orderStatus=OrderStatus(orderId=60, status='Cancelled', filled=0.0, remaining=0.0, avgFillPrice=0.0, permId=0, parentId=0, lastFillPrice=0.0, clientId=0, whyHeld='', mktCapPrice=0.0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2024, 2, 5, 21, 59, 40, 386872, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='', errorCode=0), TradeLogEntry(time=datetime.datetime(2024, 2, 5, 21, 59, 40, 389582, tzinfo=datetime.timezone.utc), status='Cancelled', message="Error 321, reqId 60: Error validating request.-'bN' : cause - Missing order exchange.", errorCode=321)], advancedError='')


# Define the SMA cross over strategy condition

In [50]:
def strategy(data,ticker):
    print('inside strategy')
    print(ticker)
    print(data)
    
    buy_condition=data['sma1'].iloc[-1]>data['sma2'].iloc[-1] and data['sma1'].iloc[-2]<data['sma2'].iloc[-2]
    sell_condition=data['sma1'].iloc[-1]<data['sma2'].iloc[-1] and data['sma1'].iloc[-2]>data['sma2'].iloc[-2]
    current_balance=int(float([v for v in ib.accountValues() if v.tag == 'AvailableFunds' ][0].value))
    if current_balance>data.close.iloc[-1]:
        if buy_condition:
            print('buy condiiton satisfied')
            close_ticker_postion(ticker)
            close_ticker_open_orders(ticker)
            trade_buy_stocks(ticker,data.close.iloc[-1])
        elif sell_condition:
            print('sell condition satisfied')
            close_ticker_postion(ticker)
            close_ticker_open_orders(ticker)
            trade_sell_stocks(ticker,data.close.iloc[-1])
        else :
            print('no condition satisfied')
    else:
        print('we dont have enough money')
        print('current balance is',current_balance,'stock price is ',data.close[-1])


# Main function calls the above and executes the strategy

In [ ]:
def main():
    ord_df=util.df(ib.reqOpenOrders())
    pos_df=util.df(ib.reqPositions())
    if pos_df is not None:
        pos_df['name']=[i.symbol for i in pos_df.contract ]
    else:
            pos_df=pd.DataFrame()
            pos_df['name']=0    

    print(pos_df)
    open_order_df=util.df(ib.openTrades())
    print(open_order_df)

    for ticker in tickers:
        print('ticker name is',ticker,'################')
        ticker_contract=ticker_contracts[ticker]
        ticker_contract=ib.qualifyContracts(ticker_contract)[0]
        hist_df=get_historical_data(ticker_contract)
        print(hist_df)
        print(hist_df.close.iloc[-1])
        capital=int(float([v for v in ib.accountValues() if v.tag == 'AvailableFunds' ][0].value))
        print(capital)
        quantity=int(capital/hist_df.close.iloc[-1])
        
        print(quantity)
        if quantity==0:
            print('we dont have enough money so we cannot trade')
            continue

        if pos_df.empty:
            print('we dont have any position')
            strategy(hist_df,ticker)


        elif len(pos_df)!=0 and ticker not in pos_df['name'].tolist():
            print('we have some position but current ticker is not in position')
            strategy(hist_df,ticker)
        elif len(pos_df)!=0 and ticker in pos_df["name"].tolist():
            print('we have some position and current ticker is in position')
            print(open_order_df)
            if pos_df[pos_df["name"]==ticker]["position"].values[0] == 0:
                print('we have current ticker in position but quantity is 0')
                strategy(hist_df,ticker)

            elif pos_df[pos_df["name"]==ticker]["position"].values[0] > 0  :
                print('we have current ticker in position and is long')
                strategy(hist_df,ticker)
            #     sell_condition=hist_df['sma1'].iloc[-1]<hist_df['sma2'].iloc[-1] and hist_df['sma1'].iloc[-2]>hist_df['sma2'].iloc[-2]
            #     current_balance=int(float([v for v in ib.accountValues() if v.tag == 'AvailableFunds' ][0].value))
            #     if current_balance>hist_df.close.iloc[-1]:
            #         if sell_condition:
            #             print('sell condition satisfied')
            #             close_ticker_postion(ticker)
            #             close_ticker_open_orders(ticker)
            #             trade_sell_stocks(ticker,hist_df.close.iloc[-1])
          


            # elif pos_df[pos_df["name"]==ticker]["position"].values[0] < 0 :
            #     print('we have current ticker in position and is short')
            #     buy_condition=hist_df['sma1'].iloc[-1]>hist_df['sma2'].iloc[-1] and hist_df['sma1'].iloc[-2]<hist_df['sma2'].iloc[-2]
            #     current_balance=int(float([v for v in ib.accountValues() if v.tag == 'AvailableFunds' ][0].value))
            #     if current_balance>hist_df.close.iloc[-1]:
            #         if buy_condition:
            #             print('buy condiiton satisfied')
            #             close_ticker_postion(ticker)
            #             close_ticker_open_orders(ticker)
            #             trade_buy_stocks(ticker,hist_df.close.iloc[-1])


# If we want to run the strategy every 1 min, we run the code below:

In [ ]:
# current_time=datetime.datetime.now()
# print(current_time)

# print(datetime.datetime.now())

# #start time
# start_hour,start_min=9,30
# #end time
# end_hour,end_min=23,30
# start_time=datetime.datetime(current_time.year,current_time.month,current_time.day,start_hour,start_min)
# end_time=datetime.datetime(current_time.year,current_time.month,current_time.day,end_hour,end_min)
# print(start_time)
# print(end_time)

# while True:
#     if datetime.datetime.now()>start_time:
#         break
#     print(datetime.datetime.now())


# candle_size=60

# while datetime.datetime.now()<end_time:
#     now = datetime.datetime.now()
#     seconds_until_next_minute = candle_size - now.second+1

#     # Sleep until the end of the current minute
#     time.sleep(seconds_until_next_minute)

#     # Run your function
#     main()

In [ ]:
main()

In [51]:
ib.disconnect()